In [3]:
# pip install mediapipe


In [4]:
import cv2
import mediapipe as mp
import math


In [5]:
# Setup MediaPipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False,
                       max_num_hands=1,
                       min_detection_confidence=0.5,
                       min_tracking_confidence=0.5)
mp_draw = mp.solutions.drawing_utils


I0000 00:00:1746194607.694101 2698088 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M4 Pro


In [6]:
# ฟังก์ชันคำนวณมุมองศาของนิ้ว
def calc_angle(a, b, c):
    ba = [a.x - b.x, a.y - b.y]
    bc = [c.x - b.x, c.y - b.y]
    dot = ba[0]*bc[0] + ba[1]*bc[1]
    norm_ba = math.hypot(*ba)
    norm_bc = math.hypot(*bc)
    cos_angle = dot / (norm_ba * norm_bc + 1e-6)
    angle = math.degrees(math.acos(max(min(cos_angle, 1.0), -1.0)))
    return angle


In [7]:
# ตรวจว่าเป็นกำมือหรือไม่
def is_fist(landmarks):
    tips_ids = [8, 12, 16, 20]  # ยกเว้นนิ้วโป้ง
    folded = 0
    for tip_id in tips_ids:
        if landmarks[tip_id].y > landmarks[tip_id - 2].y:
            folded += 1
    return folded >= 3


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [8]:
# ข้อมูลนิ้ว
FINGER_NAMES = ["Thumb", "Index", "Middle", "Ring", "Pinky"]
FINGER_TIPS = [4, 8, 12, 16, 20]
FINGER_JOINTS = {
    "Thumb": [2, 3, 4],
    "Index": [5, 6, 8],
    "Middle": [9, 10, 12],
    "Ring": [13, 14, 16],
    "Pinky": [17, 18, 20],
}


W0000 00:00:1746194607.705799 2698927 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [9]:
# เปิดกล้อง
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            landmarks = hand_landmarks.landmark

            # เช็คว่าเป็นกำมือหรือเปล่า
            if is_fist(landmarks):
                cv2.putText(frame, 'Fist Detected', (10, 40),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            else:
                cv2.putText(frame, 'Open Hand', (10, 40),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            # แสดงตำแหน่งนิ้ว
            for idx, tip_id in enumerate(FINGER_TIPS):
                finger_name = FINGER_NAMES[idx]
                tip = landmarks[tip_id]
                h, w, _ = frame.shape
                cx, cy = int(tip.x * w), int(tip.y * h)
                cv2.putText(frame, f'{finger_name}', (cx, cy - 20),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
                cv2.circle(frame, (cx, cy), 5, (255, 0, 0), -1)

            # แสดงองศา + เงื่อนไขนิ้วโป้ง
            y_offset = 80
            for name, ids in FINGER_JOINTS.items():
                a = landmarks[ids[0]]
                b = landmarks[ids[1]]
                c = landmarks[ids[2]]
                angle = calc_angle(a, b, c)

                if name == "Thumb" and angle > 70:
                    cv2.putText(frame, "Wrong", (10, y_offset),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                    y_offset += 30

                text = f"{name}: {int(angle)} deg"
                cv2.putText(frame, text, (10, y_offset),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 1)
                y_offset += 25

    cv2.imshow("Hand Tracking", frame)
    if cv2.waitKey(1) & 0xFF == 27:  # กด ESC เพื่อออก
        break

cap.release()
cv2.destroyAllWindows()


W0000 00:00:1746194607.710451 2698927 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1746194609.673004 2698927 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
